In [9]:
import numpy as np
import random
import pandas as pd
np.random.seed(42)
prices=[100,102,103,99,98,101,104,103]

In [2]:
prices=[100,102,103,99,98,101,104,103]
index_last_price=len(prices)-1
action_space_size = 3 
state_space_size = 16 
num_episodes = 10000
learning_rate = 0.2
discount_rate = 0.99
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay = 0.0003
q_table = np.zeros((state_space_size, action_space_size))


In [3]:
def get_states(prices):
    prices=prices
    states=[]
    for s in range(len(prices)):
        for b in range(2):
            states.append([s,b])
    return states

def get_next_state_reward(action,bought_price): #0=hold, 1= buy, 2=sell
    if state[1] ==0: #not yet bought stock
        if action==0:
            reward=0
            next_state=[state[0]+1,0]
            
        if action==1:
            reward=0  
            next_state=[state[0]+1,1]
            bought_price=prices[state[0]]
            
        if action==2:
            next_state=[state[0]+1,0] 
            reward=-10
            
    if state[1] ==1:   #already bought stock     
        if action==0:
            reward=0
            next_state=[state[0]+1,1]
            
        if action==1:
            reward=-10 
            next_state=[state[0]+1,1]
            
        if action==2:
            next_state=[state[0]+1,0] 
            reward=prices[state[0]]-bought_price
            bought_price=0

   
    done = True if next_state[0] == index_last_price else False  
    return next_state, reward, done, bought_price

In [4]:
states=get_states(prices)
exploration_rate=1
rewards_all_episodes=[]
for episode in range(10000):
    state = [0,0]
    done = False
    rewards_current_episode = 0
    bought_price=None
    while True: 
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[states.index(state),:]) #exploitation
        else:
            action = np.random.randint(action_space_size) #exploration
        next_state, reward, done,bought_price = get_next_state_reward(action,bought_price)

        q_table[states.index(state), action] = q_table[states.index(state), action] \ 
        * (1 - learning_rate) + learning_rate * (reward + discount_rate * \
         np.max(q_table[states.index(next_state), :]))
  
        state = next_state
        rewards_current_episode += reward
        
        if done == True: 
            break
    if exploration_rate>min_exploration_rate:
        exploration_rate -= exploration_decay
    rewards_all_episodes.append(rewards_current_episode)

In [15]:
df=pd.DataFrame(q_table,columns=["hold","buy","sell"])
df.index.name="state"
df

,hold,buy,sell
state,,,
0,6.475266,8.529761,-2.711999
1,0.000000,0.000000,0.000000
2,4.716522,7.826238,-5.812623
3,8.583993,-2.105223,7.591265
4,5.646038,4.700277,-4.322329
5,4.927442,-4.803473,8.630202
6,5.661284,4.898121,-4.202767
7,5.280969,-5.789381,1.923921
8,5.461663,5.738537,-4.462350


In [6]:
np.mean(rewards_all_episodes[-100:])

8.56

In [18]:
state=[0,0]
result=[]
while True:
    action = np.argmax(q_table[states.index(state),:])
    next_state, reward, done,bought_price = get_next_state_reward(action,bought_price)
    result.append([state[0],prices[state[0]],action,reward])
    state = next_state
    if done == True:
        break
pd.DataFrame(result,columns=["timestep","price","action", "reward"])

,timestep,price,action,reward
0,0,100,1,0
1,1,102,0,0
2,2,103,2,3
3,3,99,0,0
4,4,98,1,0
5,5,101,0,0
6,6,104,2,6
